In [ ]:
from system import *

s1 = system('x-o-o-',MMA=True)

Initializing system x-o-o-  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...


# Effective Hamiltonian as Real part

Will output $Q_n$ from expression:

$$\Delta_n = - \dfrac{\Omega^2}{\gamma} \text{Re}\{Q_n \} $$

In [ ]:
diag = 1  #input n corresponding to diag element

var('gamma','DE','De','g','g_f','Omega','v',domain='positive')
var('DEt',domain='positive',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='positive',  latex_name =r'\tilde{\Delta }_e')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')




b = s1.eff_hamiltonian_gs[diag,diag](gamma_g=0,gamma_f=gamma)

#Take only one of the compl conj terms. Switch maybe second 0 to 1
b = b.operands()[1].operands()[0] #+ b.operands()[1].operands()[3] use if obtained with sage

b = b.subs(DE=+0.5*I*gamma+DEt*gamma)
b = b.subs(De=+0.5*I*gamma+Det*gamma)

var('R_f',domain='real')#ratio  (g_f/g)^2
b = b.subs(g_f=g*sqrt(R_f))

var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
b = b.subs(v=g*sqrt(R_v))

var('R_b',domain='real')
b = b.subs(kappa_b=gamma/R_b)

var('R_c',domain='real',latex_name =r'R_c')
b = b.subs(kappa_c=gamma/R_c)

b = b.subs(g = sqrt(C)*gamma) 





b = b*gamma  #Omega has to be substituted with Omega/gamma


b = b._mathematica_().Factor()._sage_()
b = MMA_simplify(b,full=True)

show( symround(b))

# Expand linearly in the limit:   $C\rightarrow \infty$

In [ ]:
#with mathematica
q = b.subs(C=1/x)
q = MMA_simplify(q)
t = q._mathematica_().Series((x,0,2))
t_exp = t[3]
t_st = t[4]
tayl = 0
for i in range(1,len(t_exp)):
    tayl += MMA_simplify( t_exp[i]._sage_() ) / C**(i-1+Integer(t[4]))

show(symround(tayl))

#with sage
#tayl = b.taylor(C_c,infinity, 1)
#show(symround(MMA_simplify(tayl)))